In [200]:
import jieba
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2

In [181]:
# 生成vsm词袋模型
bag_of_words ={}
text_list =[]
_label = []


search_pd = pd.read_csv('./search.csv' ,header= -1)
# todo 技术方案不好的原因...?
for s in search_pd.iterrows():
    seg_list = jieba.lcut(s[1][0], cut_all=False)
    text_list.append(seg_list)
    _label.append(0)
    for seg in seg_list:
        if seg not in bag_of_words:
            bag_of_words[seg] = 1
        else:
            bag_of_words[seg] += 1


pub_pd = pd.read_csv('./pub.csv' ,header= -1)
for s in pub_pd.iterrows():
    seg_list = jieba.lcut(s[1][0], cut_all=False)
    text_list.append(seg_list)
    _label.append(1)
    for seg in seg_list:
        if seg not in bag_of_words:
            bag_of_words[seg] = 1
        else:
            bag_of_words[seg] += 1

print(bag_of_words)

{'搜': 8, '北京': 15, '夜晚': 8, '的': 32, '图': 2, '一下': 13, '晚上': 4, '壁纸': 13, '找下': 1, '想': 6, '给': 4, '手机': 1, '换': 2, '一张': 4, '电脑': 1, '找': 9, '五张': 2, '图片': 16, '蔷薇': 1, '汽车': 1, '有没有': 2, '夏天': 2, '能': 9, '不能': 2, '冬天': 2, '北邮': 6, '傍晚': 2, '我': 20, '星空': 1, '百度': 4, '天空': 1, '帮': 14, '太阳系': 1, '吗': 8, '搜索': 5, '日本': 1, '樱花': 1, '吧': 2, '在': 44, '秋天': 1, '图片吧': 1, '可以': 1, '百度搜': 2, '夜景': 1, '下': 3, '九张': 1, '十张': 1, '想要': 3, '你': 1, '上': 12, '帮个': 1, '忙': 2, '，': 25, '三张': 2, '校门口': 1, '当': 1, '发个': 5, '话题': 19, '今天': 1, '学校': 1, '检查': 1, '违禁品': 1, '请': 2, '整理': 1, '好': 5, '自己': 1, '东西': 1, '企业': 28, '版': 14, '自邮之翼': 18, '通知': 12, '所有人': 2, 'freeshare': 19, '提醒': 4, 'ch': 17, '让': 2, '他': 1, '别忘了': 3, '周三': 1, '会议': 4, '发': 8, '小组': 29, '大家': 3, '明天': 5, '下雨': 1, '记得': 2, '带伞': 1, '版自': 12, '邮之翼': 12, '内容': 5, '是': 5, '及时': 1, '提交': 1, '中期': 1, '检查报告': 1, '告诉': 5, '和': 2, 'nn': 3, '下午': 8, '早点': 1, '来': 5, '实验室': 7, '发在': 4, '就行了': 1, '取消': 1, '了': 3, '我要': 1, '上午': 1, '大会堂': 1, '见面'

In [182]:
## todo
## 运用卡方检验...来进行降维~

In [183]:
## 最后的词袋
vsm_vec = {}
row_vsm_vec = []
count = 0
for key,value in bag_of_words.items():
    sub_dic = {}
    
    # 其他的信息,比如重要性,tf-idf值之类的信息也保存再这里
    sub_dic['index'] = count
    count += 1
    vsm_vec[key] = sub_dic
    
    row_vsm_vec.append(key)
    
    sub_dic['count'] = value
    
    # todo tf-idf的值!!!
    
print(vsm_vec)

{'搜': {'index': 0, 'count': 8}, '北京': {'index': 1, 'count': 15}, '夜晚': {'index': 2, 'count': 8}, '的': {'index': 3, 'count': 32}, '图': {'index': 4, 'count': 2}, '一下': {'index': 5, 'count': 13}, '晚上': {'index': 6, 'count': 4}, '壁纸': {'index': 7, 'count': 13}, '找下': {'index': 8, 'count': 1}, '想': {'index': 9, 'count': 6}, '给': {'index': 10, 'count': 4}, '手机': {'index': 11, 'count': 1}, '换': {'index': 12, 'count': 2}, '一张': {'index': 13, 'count': 4}, '电脑': {'index': 14, 'count': 1}, '找': {'index': 15, 'count': 9}, '五张': {'index': 16, 'count': 2}, '图片': {'index': 17, 'count': 16}, '蔷薇': {'index': 18, 'count': 1}, '汽车': {'index': 19, 'count': 1}, '有没有': {'index': 20, 'count': 2}, '夏天': {'index': 21, 'count': 2}, '能': {'index': 22, 'count': 9}, '不能': {'index': 23, 'count': 2}, '冬天': {'index': 24, 'count': 2}, '北邮': {'index': 25, 'count': 6}, '傍晚': {'index': 26, 'count': 2}, '我': {'index': 27, 'count': 20}, '星空': {'index': 28, 'count': 1}, '百度': {'index': 29, 'count': 4}, '天空': {'index': 30, '

In [184]:
# 生成向量
vec_col_count = len(vsm_vec)
_list = []


for l in text_list:
    self_bag = {}
    row_vec = [0]*vec_col_count
    for wd in l:
        if wd in self_bag:
            self_bag[wd] += 1
        else:
            self_bag[wd] = 1
    for b,c in self_bag.items():
        if b in vsm_vec:
            row_vec[vsm_vec[b]['index']] = c
    _list.append(row_vec)

In [185]:
# 卡方检验得到有效特征
chi2 = SelectKBest(chi2,k=60).fit(_list, _label)

In [186]:
# 输出rank的score！！
# 排序！！

f_ip = {}
for col_index in range(0,vec_col_count):
    f_ip[col_index] = chi2.scores_[col_index]

items = f_ip.items()
it = sorted(items,key=lambda d:d[1], reverse = True)
f_ip_array = []

for key,value in it:
    f_ip_array.append(key)

f_ip_array = f_ip_array[0:100]

In [193]:
chi2_vsm_vec = {}

chi2_count = 0
for col_index in f_ip_array:
    dic = vsm_vec[row_vsm_vec[col_index]]
    dic['index'] = chi2_count
    chi2_count += 1
    chi2_vsm_vec[row_vsm_vec[col_index]] = dic



In [198]:
# 生成向量
chi2_vec_col_count = len(chi2_vsm_vec)
_chi2_list = []


for l in text_list:
    self_bag = {}
    row_vec = [0]*chi2_vec_col_count
    for wd in l:
        if wd in self_bag:
            self_bag[wd] += 1
        else:
            self_bag[wd] = 1
    for b,c in self_bag.items():
        if b in chi2_vsm_vec:
            row_vec[chi2_vsm_vec[b]['index']] = c
    _chi2_list.append(row_vec)


106


In [224]:
clf = RandomForestClassifier(n_estimators=40,oob_score= True)
clf = clf.fit(_chi2_list, _label)


# 用随机森林做~ 嗯...为什么呢...
clf.oob_score_

0.99056603773584906

In [241]:
txt = ['找一下水杯的图片','发个话题吧!','我要发一个话题','来一张长城的图片','来张壁纸','百度搜图','hi! 我要搜图']

#切词
for t in txt:
    seg_list = jieba.lcut(t, cut_all=False)

    # 生成向量
    chi2_vec_col_count = len(chi2_vsm_vec)
    _chi2_list = []

    l = seg_list

    self_bag = {}
    row_vec = [0]*chi2_vec_col_count
    for wd in l:
        if wd in self_bag:
            self_bag[wd] += 1
        else:
            self_bag[wd] = 1
    for b,c in self_bag.items():
        if b in chi2_vsm_vec:
            row_vec[chi2_vsm_vec[b]['index']] = c

    print(clf.predict([row_vec]))


[0]
[1]
[1]
[0]
[0]
[1]
[1]
